In [1]:
import os
import requests

import pandas as pd

from tqdm import tqdm
from uuid import uuid4
from sqlalchemy import create_engine

In [12]:
MKDIR_IMAGE = 'image/'

MKDIR_MOVIES = os.path.join(MKDIR_IMAGE, 'movies')
MKDIR_PEOPLE = os.path.join(MKDIR_IMAGE, 'people')

API_KEY = 'API_KEY'
URL_API = 'https://api.themoviedb.org/3/'

URL_IMAGE = 'http://image.tmdb.org/t/p/original/'

DB_HOST = '127.0.0.1'
DB_NAME = 'webfilm'
DB_USER = 'webfilm'
DB_PASS = 'nhy6&UJM'

DEFAULT_UUID = '00000000-0000-0000-0000-000000000000'

In [13]:
list_folders = [
    MKDIR_IMAGE,
    MKDIR_MOVIES,
    MKDIR_PEOPLE,
]

for x in list_folders:
    if not os.path.exists(x):
        os.makedirs(x)

In [14]:
engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')

In [15]:
list_movies = list(dict.fromkeys([
    278, 238, 629, 424, 240, 550, 510, 680, 289, 935, 274, 603, 155, 567, 769,
    1891, 11, 77, 85, 27205, 25073, 55438, 61123, 53328, 57211, 50875, 75757,
    94841, 115199, 118931, 114285, 130736, 138122, 127373, 86817, 209413, 172457,
    191293, 250376, 199740, 110, 423, 10754, 108, 31056, 8996, 109, 1600, 11502,
    22257, 29933, 5055, 659, 127, 31407, 19673, 4202, 27357, 209274, 42164, 341689,
    433086, 413992, 359749, 398818, 439502, 300665, 374471, 428493, 426264, 354282,
    460135, 426256, 406990, 411741, 336000, 455963, 293768, 414455, 395834, 192040,
    331214, 420714, 374430, 157336, 389, 74406, 77338, 73, 286217, 206514, 359364,
    200813, 8392, 203766, 127608, 25161, 167966, 423, 134777, 11, 140607, 1895,
    330459, 42979, 76180, 253150, 74849, 12180, 1894, 1893, 100402, 1771, 271110,
    284274, 106355, 197467, 197481, 13995,
]))

In [16]:
movies = pd.read_sql(
    f'SELECT movies_movie.id, title, photos_photo.id as photo_id, photos_photo.gallery_id, url '\
    f'FROM movies_movie INNER JOIN photos_photo ON movies_movie.gallery_id = photos_photo.gallery_id '\
    f'WHERE movies_movie.id IN ({", ".join([str(_) for _ in list_movies])}) '\
    f'AND photos_photo.order = 1 AND url = "{DEFAULT_UUID}";',
    engine
)
movies

,id,title,photo_id,gallery_id,url
0,53328,The Last Mountain,42237,21119,00000000-0000-0000-0000-000000000000
1,75757,America The Beautiful 2: The Thin Commandments,51391,25696,00000000-0000-0000-0000-000000000000
2,134777,The Lost Room,65395,32698,00000000-0000-0000-0000-000000000000
3,192040,Planet Earth,71791,35896,00000000-0000-0000-0000-000000000000
4,200813,The Blue Planet,72677,36339,00000000-0000-0000-0000-000000000000
5,203766,Long Way Round,72951,36476,00000000-0000-0000-0000-000000000000
6,206514,"I, Claudius",73227,36614,00000000-0000-0000-0000-000000000000
7,331214,Band of Brothers,84153,42077,00000000-0000-0000-0000-000000000000
8,374430,Black Mirror: White Christmas,87409,43705,00000000-0000-0000-0000-000000000000
9,420714,Planet Earth II,89949,44975,00000000-0000-0000-0000-000000000000


In [17]:
def download_poster(folderToSave, poster_path) -> (str, int):
    imgUrl = f'{URL_IMAGE}{poster_path}'
    name = str(uuid4())
    try:
        image = f'{folderToSave}/{name}'
        session = requests.Session()
        r = session.get(imgUrl, stream=True, verify=False)
        if r.status_code == 200:
            with open(image, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024):
                    f.write(chunk)
            return name
        else:
            raise Exception()
    except Exception as e:
        return '00000000-0000-0000-0000-000000000000'

In [18]:
for label, data  in tqdm(movies.iterrows(), total=movies.shape[0]):
    try:
        response = requests.get(f'{URL_API}movie/{data["id"]}?api_key={API_KEY}')
        response = response.json()

        movies.at[label, 'url'] = download_poster(MKDIR_MOVIES, response['poster_path'])
    except:
        pass

movies = movies[movies.url != DEFAULT_UUID]
movies

100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  7.16it/s]


,id,title,photo_id,gallery_id,url


In [47]:
for label, data  in tqdm(movies.iterrows(), total=movies.shape[0]):
    sql = f'UPDATE photos_photo SET url = \'{data["url"]}\' WHERE id = {data["photo_id"]};'

    with engine.begin() as conn:
        conn.execute(sql)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 21.52it/s]


In [20]:
list_movies_with_people = [11, 155]

In [21]:
cast = pd.read_sql(
    f'SELECT movies_person.id as person_id, name, surname, movies_person.gallery_id, photos_photo.id as photo_id, url, '\
    f'photos_photo.order FROM movies_cast INNER JOIN movies_person ON movies_cast.person_id = movies_person.id '\
    f'INNER JOIN photos_photo ON movies_person.gallery_id = photos_photo.gallery_id '\
    f'WHERE movie_id IN ({", ".join([str(_) for _ in list_movies_with_people])}) '
    f'AND photos_photo.order = 1 AND url = {DEFAULT_UUID};',
    engine
)
cast

,person_id,name,surname,gallery_id,photo_id,url,order
0,131625,Shelagh,Fraser,136489,183617,00000000-0000-0000-0000-000000000000,1
1,1012562,Anthony,Lang,193072,240200,00000000-0000-0000-0000-000000000000,1
2,1271058,Mark,Austin,245449,292577,00000000-0000-0000-0000-000000000000,1
3,1271080,Christine,Hewett,245471,292599,00000000-0000-0000-0000-000000000000,1
4,7727,Phil,Tippett,54087,101215,00000000-0000-0000-0000-000000000000,1
...,...,...,...,...,...,...,...
230,1870611,James,Fierro,393958,441086,00000000-0000-0000-0000-000000000000,1
231,170230,Tristan,Tait,150954,198082,00000000-0000-0000-0000-000000000000,1
232,1161711,Jon,Lee Brody,222739,269867,00000000-0000-0000-0000-000000000000,1
233,1655526,Nigel,Carrington,349865,396993,00000000-0000-0000-0000-000000000000,1


In [22]:
crew = pd.read_sql(
    f'SELECT movies_person.id as person_id, name, surname, movies_person.gallery_id, photos_photo.id as photo_id, url, '\
    f'photos_photo.order FROM movies_crew INNER JOIN movies_person ON movies_crew.person_id = movies_person.id '\
    f'INNER JOIN photos_photo ON movies_person.gallery_id = photos_photo.gallery_id '\
    f'WHERE movie_id IN ({", ".join([str(_) for _ in list_movies_with_people])}) '
    f'AND photos_photo.order = 1 AND url = "{DEFAULT_UUID}";',
    engine
)
crew

,person_id,name,surname,gallery_id,photo_id,url,order
0,5059,Leslie,Dilley,51605,98733,00000000-0000-0000-0000-000000000000,1
1,568911,John,Barry,175664,222792,00000000-0000-0000-0000-000000000000,1
2,5058,Roger,Christian,51604,98732,00000000-0000-0000-0000-000000000000,1
3,7753,Gilbert,Taylor,54112,101240,00000000-0000-0000-0000-000000000000,1
4,5061,John,Mollo,51607,98735,00000000-0000-0000-0000-000000000000,1
...,...,...,...,...,...,...,...
81,1341403,Richard,King,262242,309370,00000000-0000-0000-0000-000000000000,1
82,3794,Bob,Kane,50473,97601,00000000-0000-0000-0000-000000000000,1
83,198034,Bill,Finger,154596,201724,00000000-0000-0000-0000-000000000000,1
84,1356195,Jerry,Robinson,265514,312642,00000000-0000-0000-0000-000000000000,1


In [23]:
people = pd.concat([cast, crew])
people = people.drop_duplicates(['person_id'])
people

,person_id,name,surname,gallery_id,photo_id,url,order
0,131625,Shelagh,Fraser,136489,183617,00000000-0000-0000-0000-000000000000,1
1,1012562,Anthony,Lang,193072,240200,00000000-0000-0000-0000-000000000000,1
2,1271058,Mark,Austin,245449,292577,00000000-0000-0000-0000-000000000000,1
3,1271080,Christine,Hewett,245471,292599,00000000-0000-0000-0000-000000000000,1
4,7727,Phil,Tippett,54087,101215,00000000-0000-0000-0000-000000000000,1
...,...,...,...,...,...,...,...
80,1377220,Doug,Hemphill,270484,317612,00000000-0000-0000-0000-000000000000,1
82,3794,Bob,Kane,50473,97601,00000000-0000-0000-0000-000000000000,1
83,198034,Bill,Finger,154596,201724,00000000-0000-0000-0000-000000000000,1
84,1356195,Jerry,Robinson,265514,312642,00000000-0000-0000-0000-000000000000,1


In [24]:
for label, data  in tqdm(people.iterrows(), total=people.shape[0]):
    try:
        response = requests.get(f'{URL_API}person/{data["person_id"]}?api_key={API_KEY}')
        response = response.json()

        people.at[label, 'url'] = download_poster(MKDIR_PEOPLE, response['profile_path'])
    except:
        pass

people = people[people.url != DEFAULT_UUID]
people

100%|█████████████████████████████████████████| 316/316 [01:15<00:00,  4.18it/s]


,person_id,name,surname,gallery_id,photo_id,url,order
5,1271060,Jon,Berg,245451,292579,cfa36210-95fd-457c-8ce3-b1811f5fbaa5,1
11,1271104,Geoffrey,Moon,245484,292612,30c71f41-ae62-41aa-8e1b-64acdbb4362a,1
21,190453,Al,Lampert,153782,200910,7743a215-48b4-4999-815d-2d56cf8a7de1,1
27,184980,Burnell,Tucker,153091,200219,58d76b36-57a8-4190-a730-f553beca349e,1
41,159108,Larry,Ward,149005,196133,290ae3dc-b7f2-4e0b-8bd7-78f2b2724b59,1
...,...,...,...,...,...,...,...
79,1327030,Lora,Hirschberg,258313,305441,d2b3a5f5-980b-4f0b-a445-2f5960818ca4,1
80,1377220,Doug,Hemphill,270484,317612,536bb074-3e3e-4544-90e0-aa5d288d2924,1
82,3794,Bob,Kane,50473,97601,2aebfd80-8a4d-4ae5-a522-5d0ce22fbfa4,1
84,1356195,Jerry,Robinson,265514,312642,79be44bc-004c-4032-a5a4-5597978d6651,1


In [25]:
for label, data  in tqdm(people.iterrows(), total=people.shape[0]):
    sql = f'UPDATE photos_photo SET url = \'{data.url}\' WHERE id = {data.photo_id} AND photos_photo.order = 1;'

    with engine.begin() as conn:
        conn.execute(sql)

100%|█████████████████████████████████████████| 131/131 [00:01<00:00, 66.20it/s]


In [28]:
list_movies_to_backdrops = [11]

In [45]:
movies_backdrop = pd.read_sql(
    f'SELECT movies_movie.id, movies_movie.gallery_id, photos_photo.id as photo_id, photos_photo.order, url '\
    f'FROM movies_movie INNER JOIN photos_photo ON movies_movie.gallery_id = photos_photo.gallery_id '\
    f'WHERE movies_movie.id IN ({", ".join([str(_) for _ in list_movies_to_backdrops])}) '\
    f'AND photos_photo.order = 2 AND url = "{DEFAULT_UUID}";',
    engine
)

movies_backdrop

,id,gallery_id,photo_id,order,url
0,11,5,10,2,00000000-0000-0000-0000-000000000000


In [49]:
for label, data  in tqdm(movies_backdrop.iterrows(), total=movies_backdrop.shape[0]):
    try:
        response = requests.get(f'{URL_API}movie/{data.id}?api_key={API_KEY}')
        response = response.json()

        movies_backdrop.at[label, 'url'] = download_poster(MKDIR_MOVIES, response['backdrop_path'])
    except:
        pass

movies_backdrop = movies_backdrop[movies_backdrop.url != DEFAULT_UUID]
movies_backdrop

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  2.81it/s]


,id,gallery_id,photo_id,order,url
0,11,5,10,2,b23693ee-90c2-4abd-add6-b4513b8f6d10


In [50]:
for label, data  in tqdm(movies_backdrop.iterrows(), total=movies_backdrop.shape[0]):
    sql = f'UPDATE photos_photo SET url = \'{data.url}\' WHERE id = {data.photo_id};'

    with engine.begin() as conn:
        conn.execute(sql)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 19.78it/s]


In [8]:
jsonSTR ="""
[
    {
        "id": 100402,
        "title": "Captain America: The Winter Soldier",
        "release_date": "2014-03-20",
        "gallery": 29420,
        "wr": "6.220605148974762",
        "poster_url": {
            "url": "00000000-0000-0000-0000-000000000000",
            "order": 1
        }
    },
    {
        "id": 1771,
        "title": "Captain America: The First Avenger",
        "release_date": "2011-07-22",
        "gallery": 1111,
        "wr": "6.056101286887946",
        "poster_url": {
            "url": "00000000-0000-0000-0000-000000000000",
            "order": 1
        }
    },
    {
        "id": 271110,
        "title": "Captain America: Civil War",
        "release_date": "2016-04-27",
        "gallery": 39569,
        "wr": "5.909209119689295",
        "poster_url": {
            "url": "00000000-0000-0000-0000-000000000000",
            "order": 1
        }
    },
    {
        "id": 284274,
        "title": "Iron Man & Captain America: Heroes United",
        "release_date": "2014-07-29",
        "gallery": 40151,
        "wr": "5.052548473097545",
        "poster_url": {
            "url": "00000000-0000-0000-0000-000000000000",
            "order": 1
        }
    },
    {
        "id": 106355,
        "title": "Captain America",
        "release_date": "1944-02-05",
        "gallery": 30068,
        "wr": "5.05112568785623",
        "poster_url": {
            "url": "00000000-0000-0000-0000-000000000000",
            "order": 1
        }
    },
    {
        "id": 197467,
        "title": "Captain America",
        "release_date": "1979-01-19",
        "gallery": 36140,
        "wr": "5.050620765606826",
        "poster_url": {
            "url": "00000000-0000-0000-0000-000000000000",
            "order": 1
        }
    },
    {
        "id": 197481,
        "title": "Captain America II: Death Too Soon",
        "release_date": "1979-11-23",
        "gallery": 36141,
        "wr": "5.050226964793841",
        "poster_url": {
            "url": "00000000-0000-0000-0000-000000000000",
            "order": 1
        }
    },
    {
        "id": 13995,
        "title": "Captain America",
        "release_date": "1990-12-14",
        "gallery": 6321,
        "wr": "5.045994656238274",
        "poster_url": {
            "url": "00000000-0000-0000-0000-000000000000",
            "order": 1
        }
    }
]
"""

In [9]:
import json
aList = json.loads(jsonSTR)
print(', '.join(
    [str(_['id']) for _ in aList])
)

100402, 1771, 271110, 284274, 106355, 197467, 197481, 13995


{'adult': False,
 'backdrop_path': '/zqkmTXzjkAgXmEWLRsY4UpTWCeo.jpg',
 'belongs_to_collection': {'id': 10,
  'name': 'Star Wars Collection',
  'poster_path': '/tdQzRSk4PXX6hzjLcQWHafYtZTI.jpg',
  'backdrop_path': '/d8duYyyC9J5T825Hg7grmaabfxQ.jpg'},
 'budget': 11000000,
 'genres': [{'id': 12, 'name': 'Adventure'},
  {'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.starwars.com/films/star-wars-episode-iv-a-new-hope',
 'id': 11,
 'imdb_id': 'tt0076759',
 'original_language': 'en',
 'original_title': 'Star Wars',
 'overview': 'Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire.',
 'popularity': 80.284,
 'poster_path': '/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg',
 'production_companies': [{'id